In [1]:
# Import the basic spark library
from pyspark.sql import SparkSession

# Create an entry point to the PySpark Application
# spark is our connection to the database 
spark = SparkSession.builder \
      .master("local") \
      .appName("MyFirstSparkApplication") \
      .getOrCreate()
# master contains the URL of your remote spark instance or 'local'

In [3]:
# Load a DataFrame
df = spark.read.option("header", True).option("delimiter", "|").option("inferSchema",True).csv("dataset/author.csv")

# Print detected 
df.printSchema()

df.show()

root
 |-- name: string (nullable = true)
 |-- orcid: string (nullable = true)
 |-- month_of_birth: integer (nullable = true)
 |-- year_of_birth: integer (nullable = true)
 |-- mail: string (nullable = true)

+--------------------+-------------------+--------------+-------------+--------------------+
|                name|              orcid|month_of_birth|year_of_birth|                mail|
+--------------------+-------------------+--------------+-------------+--------------------+
|       Gilles Guette|9404-5450-4767-7840|             6|         1985|    iwAaxsL@duck.com|
|      Renaud Pacalet|2886-4313-9155-9063|            12|         1952|      mAgIX@duck.com|
|       Hak-Keung Lam|2629-3989-5026-3274|             6|         1942|     maGyVg@duck.com|
|Sebastian J. F. F...|0000-0002-3553-5131|             7|         1977|  utqmZOW@outlook.it|
|      Cristiana Amza|6548-4280-5081-9705|             6|         1949|   WcmIhU@outlook.it|
|   Afshan Nourizadeh|2570-5168-4862-2639|      

<h4>Resilient Distributed Dataset (RDD)</h4>
<ul>
    <li>Fault tolerant</li>
    <li>Resilient</li>
    <li>Immutable</li>
    <li>Partitioned</li>
</ul>

<h4>Data Upload</h4>

In [ ]:
# Upload data from a list  
data = [("Margherita", 5.95, ["Tomato Sauce", "Mozzarella Cheese", "Basil"]),
        ("Calzone", 7.95, ["Tomato Sauce", "Mozzarella Cheese", "Prosciutto Cotto"])]

# Create an RDD
rdd = spark.sparkContext.parallelize(data)

In [ ]:
# Upload list from a file
rdd_2 = spark.sparkContext.textFile("menu.txt")

<h4>Dataframe Creation</h4>

In [ ]:
# Create a Dataframe
df_data = [("Margherita", 5.95, ["Tomato Sauce", "Mozzarella Cheese", "Basil"]),
        ("Calzone", 7.95, ["Tomato Sauce", "Mozzarella Cheese", "Prosciutto Cotto"]),
        ("Diavola", 5.95, ["Tomato Sauce", "Mozzarella Cheese", "Spicy Salame"]),
        ("Prosciutto", 7.95, ["Tomato Sauce", "Mozzarella Cheese", "Prosciutto Cotto"]),
        ("Speck & Brie", 7.95, ["Tomato Sauce", "Mozzarella Cheese", "Speck", "Brie"]),
        ("Tonno & Cipolle", 7.95, ["Tomato Sauce", "Mozzarella Cheese", "Tuna", "Onions"]),
        ("Fries", 3.95, ["Potatoes"])]
        
columns = ["Pizza Name", "Price", "Ingredients"]
df = spark.createDataFrame(data = df_data, schema = columns)

In [ ]:
# Show the first 20 elements of a dataframe
df.show()

In [ ]:
# Load a DataFrame
df = spark.read.option("header", True).option("delimiter", ";").csv("menu_csv.txt")

# Print detected 
df.printSchema()

df.show()

<h4>Dataframes from RDDs</h4>

In [ ]:
# Transform the RDD into a Dataframe
df_from_rdd = rdd.toDF()

# Print the schema of the Dataframe
df_from_rdd.printSchema()

In [ ]:
#Transform the RDD into a Dataframe, specifying the columns
columns = ["Pizza Name", "Price", "Ingredients"]
df_from_rdd = rdd.toDF(columns)
df_from_rdd.printSchema()

In [ ]:
df_2_from_rdd = spark.createDataFrame(rdd).toDF(*columns)
df_from_rdd.printSchema()

<h4>Custom Dataframe</h4>

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType, ArrayType

#Createe the schema using StructField(Name, Type, Nullable)
schema = StructType([ \
    StructField("Pizza Name", StringType(), True), \
    StructField("Price", FloatType(), True), \
    StructField("Ingredients", ArrayType(StringType()), True) \
])
 
df = spark.createDataFrame(data = df_data, schema = schema)
df.printSchema()
df.show(truncate=False)

<h4>Organizing Data</h4>

In [ ]:
# Sorting depending on the fields (default = ascending order)
df.sort("Price").show(truncate = False)

In [ ]:
from pyspark.sql.functions import col
# Sorting depending on the fields
df.sort(col("Price"), col("Pizza Name")).show(truncate = False) # or use df.Price

In [ ]:
# Sorting using orderBy
df.orderBy(col("Price"), col("Pizza Name")).show(truncate = False) # in case of df.Pizza Name i cant use it
# so i use the function col("Pizza Name")

In [ ]:
# Expliciting the sorting (work the same with orderBy)
df.sort(col("Price").asc(), col("Pizza Name").desc()).show(truncate = False)

In [ ]:
# We could also use raw SQL
# No spoilers -> We'll see how to use it later on

<h4>Explode Arrays in Individual Rows</h4>

In [ ]:
from pyspark.sql.functions import explode # its like unwind on mongodb 
# it splut the array into a lot of row with only one value of the array

exploded_df = df.select(col("Pizza Name"), df.Price, explode(df.Ingredients))
exploded_df.printSchema()
exploded_df.show(truncate = False)

In [ ]:
# How can we rename a column?
exploded_df = exploded_df.withColumnRenamed("col", "Ingredient").printSchema()